In [1]:
import xml.sax
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
filename = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
sergey = '../Sergey_brin.xml'

In [ ]:
def cleanupText(arr):
    for stats in arr:
        stats = re.sub(r'http[^\ ]*\ ',r' ',stats)
        stats = word_tokenize(stats)

In [3]:
class RegexExp():
    def __init__(self):
        pass
    def findUsefulText(self, info):
        stats = info.split('\n')
        refs = []
        extlinks = []
        cat = []
        for l in stats:
            if re.search(r'ref',l):
                x = re.sub(r'.*title[\ ]*=[\ ]*([^\|]*).*',r'\1',l)
                refs.append(x)
            if re.search(r'\[\[category',l):
                x = re.sub(r'\[\[category:(.*)\]\]',r'\1',l)
                cat.append(x)
            if re.match(r'\*[\ ]*\[',l):
                extlinks.append(l)
        
        return (refs,extlinks,cat)
    
    def textHandling(self,ID,text,title):
        text = text.lower()
        cleaned_text = text.split('==references==')
#         print(data)
        if len(cleaned_text) == 1:
            cleaned_text = text.split('== references== ')
        if len(cleaned_text) == 1:
            references = []
            links = []
            categories = []
        else:
            useful = cleaned_text[1]
            (refs,extlink,cat) = self.findUsefulText(useful)
        
            

In [4]:
class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ''
        self.title = ''
        self.ID = ''
        self.idFlag = 0
        self.text = ""
        
    def startElement(self,tag,attributes):
        self.CurrentData = tag
            
    def endElement(self,tag):
        if self.CurrentData == 'title':
            pass
#             print(self.title)
        elif self.CurrentData == 'text':
            pass
#             print(self.text)
        
        
        
        if tag == 'page':
            reg = RegexExp()
            reg.textHandling(self.ID,self.text,self.title)
            self.CurrentData = ''
            self.title = ''
            self.ID = ''
            self.idFlag = 0
            self.text = ''
            
        
        
    def characters(self,content):
        if self.CurrentData == 'title':
            self.title += content
        elif self.CurrentData == 'text':
            self.text += content
        elif self.CurrentData == 'id' and self.idFlag == 0:
            self.ID = content
            self.idFlag = 1

In [5]:
class Parser():
    def __init__(self,filename):
        self.parser = xml.sax.make_parser()
        self.parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        self.handler =  XMLHandler()
        self.parser.setContentHandler(self.handler)
        self.parser.parse(filename)

In [6]:
parser = Parser(filename)